# PyTorch Lightning baseline template

This notebook is meant to act as a template to train and use a simple regression model to define a baseline that can be compared with a DS application.

It focuses on the concept of defining a PyTorch model, a PyTorch lightning training loop and the deffinition of metrics of performance.

This notebook assumes familiarity with the concepts of datasets and dataloaders contained in the **_0_dataset_dataloader_template.ipynb_**

## Set your cuda visible device

**IMPORTANT:** Since we are sharing resources, please make sure that the cuda visible device you put here is the one assigned to your team and your machine.   

In [2]:
import torch.multiprocessing as mp
mp.set_sharing_strategy("file_system")

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

Here we initalize variables related to Weights and Biases, our online logging system to ensure they are user specific

In [4]:
# Make sure wandb logs are stored in a user-specific directory
# Set writable directories
#os.environ["WANDB_DIR"] = "./wandb/wandb_logs"
#os.environ["WANDB_CACHE_DIR"] = "./wandb/wandb_cache"
#os.environ["WANDB_CONFIG_DIR"] = "./wandb/wandb_config"
# Optional:
#os.environ["TMPDIR"] = "./wandb/wandb_tmp"

# Ensure directories exist (optional, wandb usually creates them)
#os.makedirs(os.environ["WANDB_DIR"], exist_ok=True)
#os.makedirs(os.environ["WANDB_CACHE_DIR"], exist_ok=True)
#os.makedirs(os.environ["WANDB_CONFIG_DIR"], exist_ok=True)
#os.makedirs(os.environ["TMPDIR"], exist_ok=True)

In [5]:

import sys
from torch.utils.data import DataLoader

import torch
import yaml

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger, WandbLogger

# Append base path.  May need to be modified if the folder structure changes.
# It gives the notebook access to the wokshop_infrastructure folder.
sys.path.append("../../")
 
# Append Surya path. May need to be modified if the folder structure changes.
# It gives the notebook access to surya's release code.
sys.path.append("../../Surya")

from surya.utils.data import build_scalers  # Data scaling utilities for Surya stacks

torch.set_float32_matmul_precision('medium')



## Download scalers
Surya input data needs to be scaled properly for the model to work and this cell downloads the scaling information.

- If the cell below fails, try running the provided shell script directly in the terminal.
- Sometimes the download may fail due to network or server issues—if that happens, simply re-run the script a few times until it completes successfully.

In [6]:
!sh download_scalers.sh

==> Checking assets directory at: /home/sophiemusset/surya_workshop/downstream_apps/templatecopy/assets
==> Downloading scalers.yaml from nasa-ibm-ai4science/core-sdo
Fetching 1 files: 100%|██████████████████████████| 1/1 [00:00<00:00, 795.13it/s]
/home/sophiemusset/surya_workshop/downstream_apps/templatecopy/assets
✓ Done. File is located at: /home/sophiemusset/surya_workshop/downstream_apps/templatecopy/assets/scalers.yaml


## Load configuration

Surya was designed to read a configuration file that defines many aspects of the model
including the data it uses we use this config file to set default values that do not
need to be modified, but also to define values specific to our downstream application

In [7]:
# Configuration paths - modify these if your files are in different locations
config_path = "./configs/config.yaml"

# Load configuration
print("📋 Loading configuration...")
try:
    config = yaml.safe_load(open(config_path, "r"))
    config["data"]["scalers"] = yaml.safe_load(open(config["data"]["scalers_path"], "r"))
    print("✅ Configuration loaded successfully!")
except FileNotFoundError as e:
    print(f"❌ Error: {e}")
    print("Make sure config.yaml exists in your current directory")
    raise

scalers = build_scalers(info=config["data"]["scalers"])

📋 Loading configuration...
✅ Configuration loaded successfully!


## Define Downstream (DS) datasets

This child class takes as input all expected HelioFM parameters, plus additonal parameters relevant to the downstream application.  Here we focus in particular to the DS index and parameters necessary to combine it with the HelioFM index.

Another important component of creating a dataset class for your DS is normalization.  Here we use a log normalization on xray flux that will act as the output target.  Making log10(xray_flux) strictly positive and having 66% of its values between 0 and 1

In this case we will define both a training and a validation dataset using the indices pointed at in the config

**_Important:  In this notebook we sets max_number_of_samples=6 to potentially avoid going through the whole dataset as we explore it.  Keep in mind this for the future in case the database seems smaller than you expect_**


In [8]:
from downstream_apps.templatecopy.datasets.jet_dataset import SolarJetDataset

In [9]:
train_dataset = SolarJetDataset(
    #### All these lines are required by the parent HelioNetCDFDataset class
    index_path=config["data"]["train_data_path"],
    time_delta_input_minutes=config["data"]["time_delta_input_minutes"],
    time_delta_target_minutes=config["data"]["time_delta_target_minutes"],
    n_input_timestamps=config["model"]["time_embedding"]["time_dim"],
    rollout_steps=config["rollout_steps"],
    channels=config["data"]["channels"],
    drop_hmi_probability=config["drop_hmi_probablity"],
    use_latitude_in_learned_flow=config["use_latitude_in_learned_flow"],
    scalers=scalers,
    phase="train",
    s3_use_simplecache = False,
    s3_cache_dir= "/tmp/helio_s3_cache",    
    #### Put your donwnstream (DS) specific parameters below this line
    return_surya_stack=True,
    max_number_of_samples=6,
    ds_flare_index_path="jet_cat_surya_v2.csv",
    ds_time_column="start_time",
    ds_time_tolerance = "0.5h",
    ds_match_direction = "forward",  
    ds_fov_size_pix = 320,  
)

# The Validation dataset changes the index we read
val_dataset = SolarJetDataset(
    #### All these lines are required by the parent HelioNetCDFDataset class
    index_path=config["data"]["valid_data_path"],  #<---------------- different index path
    time_delta_input_minutes=config["data"]["time_delta_input_minutes"],
    time_delta_target_minutes=config["data"]["time_delta_target_minutes"],
    n_input_timestamps=config["model"]["time_embedding"]["time_dim"],
    rollout_steps=config["rollout_steps"],
    channels=config["data"]["channels"],
    drop_hmi_probability=config["drop_hmi_probablity"],
    use_latitude_in_learned_flow=config["use_latitude_in_learned_flow"],
    scalers=scalers,
    s3_use_simplecache = False,
    s3_cache_dir= "/tmp/helio_s3_cache",    
    #### Put your donwnstream (DS) specific parameters below this line
    return_surya_stack=True,
    max_number_of_samples=6,
    ds_flare_index_path="jet_cat_surya_v2.csv",
    ds_time_column="start_time",
    ds_time_tolerance = "0.5h",
    ds_match_direction = "forward",  
    ds_fov_size_pix = 320,   
)

We also intialize separate training and validation dataloaders.   Since we are working in a shared environment.  Using multiprocessing_context="spawn" helps avoid lockups.

In [10]:
batch_size = 2

train_data_loader = DataLoader(
                dataset=train_dataset,
                batch_size=batch_size,
                shuffle=True,
                num_workers=4,
                multiprocessing_context="spawn",
                persistent_workers=True,
                pin_memory=True,
            )

val_data_loader = DataLoader(
                dataset=val_dataset,
                batch_size=batch_size,
                num_workers=4,
                multiprocessing_context="spawn",
                persistent_workers=True,
                pin_memory=True,
            )

## Define simple baseline model

Defining a simple baseline is important to understand what value is bringing the AI model to the problem.  

It is always very good to have a very simple baseline model.  Ideally one that cannot overfit the data.  This is a very good way of really measuring the value added of complex models.   Classical machine learning excels here:

- Regressions and logistic regressions.
- Climatological averages.
- Persistance.
- Simple transformations.

Simple models avoid excesively optimistic assessments of the capatiblities of a complex models and for many problems are actually remarkably hard to beat.

In this example we define a simple regression acting on the intensity of each channel.  Note that we invert the normalization to deal with strictly positive quantities.  As with the dataset we will be importing the model from a module so that we can use it within training scripts later on.

In [11]:
from downstream_apps.template.models.simple_baseline import RegressionFlareModel

We can now test that this model manipulates a batch as expected and returns an estimate of flare intensity

Note that the simple regression model definition requires knowing the number of channels and timesteps so here we pull that information from the configuration intializing the model.

In [12]:
n_input_timestamps = config["model"]["time_embedding"]["time_dim"]
n_channels = len(config["data"]["channels"])

model = RegressionFlareModel(n_input_timestamps*n_channels, config["data"]["channels"], scalers)

Now we can pass the input stack 'ts' to the model to transform it into our regression output.   Note that since this model has not been trained and was initalized randomly.  The output here has no real meaning.  It only acts as a test that our model forward doesn't have dimension problems.

Dimension problemns are the dominant source of error in this kind of work.

Note that our output has now the size of our batch.

In [13]:
batch = next(iter(train_data_loader))
batch

{'ts': tensor([[[[[ 4.7602e-01,  5.8040e-01, -6.1669e-02,  ...,  9.2633e-01,
              8.9322e-01,  4.9555e-01],
            [ 3.8441e-02,  4.6576e-01, -1.2576e-01,  ...,  7.5505e-01,
              6.9330e-01,  3.9282e-01],
            [-9.5171e-02,  1.0838e-01, -1.4298e-01,  ...,  9.1878e-01,
              6.1846e-01,  5.3820e-02],
            ...,
            [ 4.0271e+00,  3.5948e+00,  4.1655e+00,  ...,  5.8170e-03,
             -8.4515e-02, -1.2998e-01],
            [ 4.1206e+00,  4.0306e+00,  3.7645e+00,  ...,  4.2291e-01,
              1.1373e-01, -2.6720e-02],
            [ 4.0716e+00,  4.2173e+00,  3.8181e+00,  ..., -9.2615e-02,
             -1.5617e-01, -2.6910e-01]]],
 
 
          [[[ 1.1378e-01, -1.3050e-01, -4.0767e-01,  ...,  3.5390e+00,
              2.8147e+00,  2.1812e+00],
            [ 7.7825e-03,  2.9774e-02, -4.8910e-02,  ...,  2.5293e+00,
              1.7423e+00,  1.8072e+00],
            [-2.5801e-02, -1.1863e-01, -7.9712e-02,  ...,  2.5222e+00,
            

In [ ]:
batch.keys()

dict_keys(['ts', 'time_delta_input', 'forecast', 'lead_time_delta', 'ds_index', 'jet'])

In [15]:
batch["forecast"]

tensor([[[[[-3.2807e-01, -2.9444e-01, -5.2382e-01,  ..., -5.4036e-01,
            -5.4036e-01, -5.4036e-01],
           [-3.9086e-01, -3.9618e-01, -4.3914e-01,  ..., -5.4036e-01,
            -5.4036e-01, -5.4036e-01],
           [-3.3759e-01, -4.3269e-01, -3.8386e-01,  ..., -5.4036e-01,
            -5.4036e-01, -5.4036e-01],
           ...,
           [-1.1253e-01, -4.9463e-01, -5.0804e-01,  ..., -4.0873e-01,
            -3.4127e-01, -5.4036e-01],
           [-1.2001e-01, -1.5518e-01, -4.3541e-01,  ..., -3.6245e-01,
            -1.6629e-01, -5.4036e-01],
           [-4.0943e-01, -4.4639e-01, -4.2893e-01,  ..., -5.4036e-01,
            -5.4036e-01, -5.4036e-01]]],


         [[[-6.3085e-01, -6.3085e-01, -6.1305e-01,  ..., -7.0938e-01,
            -7.0938e-01, -7.0938e-01],
           [-6.3085e-01, -6.3085e-01, -5.9222e-01,  ..., -6.8224e-01,
            -6.8907e-01, -7.0938e-01],
           [-7.0938e-01, -7.0113e-01, -6.0761e-01,  ..., -6.8224e-01,
            -6.8907e-01, -7.0938e-01],

In [19]:
import numpy as np
np.shape(batch["forecast"])

torch.Size([2, 13, 1, 4096, 4096])

In [20]:
#batch = next(iter(train_data_loader))
output = model.forward(batch['ts'])[:,0]  # Get rid of singleton dimension
output

tensor([-0.4477, -0.5655], grad_fn=<SelectBackward0>)

## Define your metrics

Metrics are a very important part of training AI models.   They provide your models with the quantitification of error, which in turn shifts the weights towards better pefrorming models.  They also provide a way for you to monitor performance, identify overfitting, and quantify value added. 

We now initialize the metrics class which allows you to control what metrics do you want to use as "loss" (i.e. the metrics that backpropagate through your model) and which ones for monitoring performance.  As with other components, this takes the form of a loaded module that can be later use in a training script

In [21]:
from downstream_apps.template.metrics.template_metrics import FlareMetrics

In [22]:
train_loss_metrics = FlareMetrics("train_loss")
train_evaluation_metrics = FlareMetrics("train_metrics")
validation_evaluation_metrics = FlareMetrics("val_metrics")

Now they can be evaluated in our model's output and our ground truth.   First the loss that actually will backpropagate, in this case Mean Squared Errror

In [23]:
#train_loss_metrics(output, batch["forecast"])
train_loss_metrics(output, batch["jet"])

printing preds:
tensor([-0.4477, -0.5655], grad_fn=<SelectBackward0>)
printing target:
tensor([1, 1])


({'mse': tensor(2.2734, grad_fn=<MseLossBackward0>)}, [1])

Then a training evaluation that will not backpropagate and inform our model, but that we can keep an eye on. Note that reporting lots of metrics during training will slow the training process.  I'm including it her as an example, but oftentimes is better to put the diagnostics only in the validation evaluation metrics.

Here we are caclulating the Root Relative Squared Error https://lightning.ai/docs/torchmetrics/stable/regression/rse.html 

A value below one means the prediction is better than predicting the average.  It is unlikely that this metric will be lower than one with a randomly initialized model

In [24]:
train_evaluation_metrics(output, batch["jet"])

({'rrse': tensor(6175.8018)}, [1])

In the validation evaluation metrics we report both MSE and RRSE

In [25]:
validation_evaluation_metrics(output, batch["jet"])

({'mse': tensor(2.2734), 'rrse': tensor(6175.8018)}, [1, 1])

## Define your PyTorch ligthning module

In this workshop we will use PyTorch lightning to train our models.  PyTorch lighting reduces the amount of code required to implement a training loop in comparison to PyTorch (at the expense of control and versatility).  

Opening the FlareLightningModule shows a simple Lightning model implementation.  It consists of:

- An initialization of the class (metrics, model, and learning rate).
- The forward code that runs evaluation of the model.
- Training and validation steps.
- Configuration of optimizers.

In [34]:
from downstream_apps.template.lightning_modules.pl_simple_baseline_jet import FlareLightningModule

## Set your global seeds

Since training AI models generally uses stochastic gradient descent, it is a good idea to fix your random seeds so that your training exercise is reproducible.    

In [35]:
pl.seed_everything(42, workers=True)

Seed set to 42


42

## Intialize Lightning module

Now we properly initalize the Lightning module to enable training, including passing the dictionary of metrics

In [37]:
metrics = {'train_loss': train_loss_metrics,
           'train_metrics': train_evaluation_metrics,
           'val_metrics': validation_evaluation_metrics}

learning_rate = 1e-3
lit_model = FlareLightningModule(model, metrics, lr=learning_rate, batch_size=batch_size)

## Logging

In order to properly compare experiments against each other, it is very useful to log evaluation metrics in a place where they can be compared against other training runs.  In this workshop we will use Weights and Biases (WandB). 

The first time you run WandB in a machine it will ask you to login to WandB.  You should have received an invitation to our project.  In order to login you must:

- Select option 2 (existing account).   In VScode the dialog opens a box at the top of your screen.
- Click on get API Key (this will open a browser).
- Generate API Key.
- Paste it in the dialog box at the top of your VSCode

In [38]:
project_name = "solar_jet_regression"
run_name = "baseline_experiment_1"

wandb_logger = WandbLogger(
    entity="surya_handson",
    project=project_name,
    name=run_name,
    log_model=False,
    save_dir="./wandb/wandb_tmp",
)

csv_logger = CSVLogger("runs", name="simple_flare")

## Initialize trainer

With the loggers done, now the trainer needs to be defined.  The trainer defines several properties of your training run. Here we define:

- The max number of epochs (one epoch represents your model seeing your entire training dataset).
- Define where the training run will take place (auto uses the GPU if possible, if not, CPU).
- The loggers.
- The callbacks (here we save the model with the lowest validation loss).
- Logging frequency (because we are working with a small dataset it needs to be small).

In [39]:
max_epochs = 2

# -------------------------------------------------------------------------
# Trainer
# -------------------------------------------------------------------------
trainer = pl.Trainer(
    max_epochs=max_epochs,
    accelerator="auto",
    devices="auto",
    logger=[wandb_logger, csv_logger],
    callbacks=[
        ModelCheckpoint(
            monitor="val_loss",
            mode="min",
            save_top_k=1,
        )
    ],
    log_every_n_steps=2,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


## Fit the model

Finally we fit the model.  We pass the Lighting module, and our dataloaders.

In [40]:
trainer.fit(lit_model, train_data_loader, val_data_loader)

/opt/anaconda3/envs/surya_ws/lib/python3.12/site-packages/pytorch_lightning/loggers/wandb.py:400: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [4]

  | Name  | Type                 | Params | Mode | FLOPs
--------------------------------------------------------------
0 | model | RegressionFlareModel | 14     | eval | 0    
--------------------------------------------------------------
14        Trainable params
0         Non-trainable params
14        Total params
0.000     Total estimated model params size (MB)
0         Modules in train mode
2         Modules in eval mode
0         Total Flops


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

printing preds:
tensor([[-0.2899],
        [-0.8111]], device='cuda:0')
printing target:
tensor([[1.],
        [1.]], device='cuda:0')
printing preds:
tensor([[-0.8225],
        [-0.6853]], device='cuda:0')
printing target:
tensor([[1.],
        [1.]], device='cuda:0')


/opt/anaconda3/envs/surya_ws/lib/python3.12/site-packages/pytorch_lightning/loops/fit_loop.py:534: Found 2 module(s) in eval mode at the start of training. This may lead to unexpected behavior during training. If this is intentional, you can ignore this warning.


Training: |          | 0/? [00:00<?, ?it/s]

...

printing preds:
tensor([[-0.2828],
        [-0.7822]], device='cuda:0')
printing target:
tensor([[1.],
        [1.]], device='cuda:0')
printing preds:
tensor([[-0.7902],
        [-0.6595]], device='cuda:0')
printing target:
tensor([[1.],
        [1.]], device='cuda:0')
printing preds:
tensor([[-0.7847],
        [-0.2858]], device='cuda:0')
printing target:
tensor([[1.],
        [1.]], device='cuda:0')
printing preds:
tensor([[-0.4370],
        [-0.4308]], device='cuda:0', grad_fn=<AddmmBackward0>)
printing target:
tensor([[1.],
        [1.]], device='cuda:0')
printing preds:
tensor([[-0.4426],
        [-0.5363]], device='cuda:0', grad_fn=<AddmmBackward0>)
printing target:
tensor([[1.],
        [1.]], device='cuda:0')
printing preds:
tensor([[-0.4427],
        [-0.4568]], device='cuda:0', grad_fn=<AddmmBackward0>)
printing target:
tensor([[1.],
        [1.]], device='cuda:0')


Validation: |          | 0/? [00:00<?, ?it/s]

printing preds:
tensor([[-0.2757],
        [-0.7536]], device='cuda:0')
printing target:
tensor([[1.],
        [1.]], device='cuda:0')


`Trainer.fit` stopped: `max_epochs=2` reached.


printing preds:
tensor([[-0.7582],
        [-0.6340]], device='cuda:0')
printing target:
tensor([[1.],
        [1.]], device='cuda:0')
printing preds:
tensor([[-0.7531],
        [-0.2763]], device='cuda:0')
printing target:
tensor([[1.],
        [1.]], device='cuda:0')


## Conclusion

With this we have now integrated our dataset, dataloaders, metrics, and baseline into an end-2-end training loop.  The next step is to substitute the simple model with Surya.